# Projet 7 Mission 3

Cette partie génère un grand nombre de lignes pour notre tableau, le fichier est par conséquence très long à exécuter.


In [1]:

#import des librairies
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import seaborn as sns
import fonctions as f
pd.set_option('display.float_format','{:.3f}'.format)
%matplotlib inline 


In [2]:

# Import des donées
df_p7 = pd.read_csv("data/clean/set_dataProjet7.csv")
df_info = pd.read_csv("data/clean/set_infoGeneral.csv")
df_coef = pd.read_csv('data/CoefElast/coefElast.csv')


In [3]:

#Tableau que nous avons créé à partir des informations du document texte fournis dans les documents.
df_coef
df_info['Coef_Elast'] = 0


In [4]:

df_coef.Region.values


array(['Europe', 'USA', 'Asia', 'Latin America/Africa'], dtype=object)

In [5]:

#Création et initialisation de la variable du coeficient d'élasticité
df_info['Coef_Elast'] = 0

#Mise en place des bonnes valeur pour chaqu'une des régions.
df_info.loc[df_info[df_info['Region'].isin(["East Asia & Pacific","South Asia"])].index,"Coef_Elast"] = df_coef[df_coef['Region'] == 'Asia'].coef.values[0]
df_info.loc[df_info[df_info['Region'].isin(["Latin America & Caribbean","Sub-Saharan Africa","Middle East & North Africa"])].index,"Coef_Elast"] = df_coef[df_coef['Region'] == 'Latin America/Africa'].coef.values[0]
df_info.loc[df_info[df_info['Region'].isin(["Europe & Central Asia"])].index,"Coef_Elast"] = df_coef[df_coef['Region'] == 'Europe'].coef.values[0]
df_info.loc[df_info[df_info['Region'].isin(["North America","South Asia"])].index,"Coef_Elast"] = df_coef[df_coef['Region'] == 'USA'].coef.values[0]


In [6]:

df_info.head()


,Country_Name,Country_Code,Population,Region,Income_Groupe,Gini,gdpppp,Coef_Elast
0,Albania,ALB,2947314.000,Europe & Central Asia,Upper middle income,32.141,7297.000,0.400
1,Argentina,ARG,40080160.000,Latin America & Caribbean,Upper middle income,46.525,13220.000,0.660
2,Armenia,ARM,2907618.000,Europe & Central Asia,Upper middle income,28.030,5611.000,0.400
3,Austria,AUT,8321496.000,Europe & Central Asia,High income,29.533,36193.000,0.400
4,Azerbaijan,AZE,8763400.000,Europe & Central Asia,Upper middle income,32.903,8101.000,0.400


In [7]:

df_info.Coef_Elast.unique()


array([0.4 , 0.66, 0.5 ])

In [8]:

df_info.isna().sum()


Country_Name     0
Country_Code     0
Population       0
Region           0
Income_Groupe    0
Gini             0
gdpppp           0
Coef_Elast       0
dtype: int64

Nous allons créer un autre tableau que nous allons utiliser par la suite, il nous faudra avoir certaines informations.

- Pays
- Classe économique
- income moyen
- Coefficient d'élasticité

Nous nous servirons du dataframe avec les informations par quartiles pour chaq'un des pays.

In [9]:

df_p7.head()


,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.898,7297.000
1,ALB,2008,2,100,916.662,7297.000
2,ALB,2008,3,100,1010.916,7297.000
3,ALB,2008,4,100,1086.908,7297.000
4,ALB,2008,5,100,1132.700,7297.000


Nous avons donc le datafarame demander question 9

In [10]:

#On crée un dataframe qui ressemble au final, mais ici nous allons lui assigner les probabilités pour chaque cas
df_ProbaCoefDefault = pd.DataFrame({'Ci_Child': np.arange(1,101)})
df_ProbaCoefDefault = pd.concat([df_ProbaCoefDefault] * 100 )
df_ProbaCoefDefault = df_ProbaCoefDefault.sort_values(by='Ci_Child')
df_ProbaCoefDefault = df_ProbaCoefDefault.reset_index()
df_ProbaCoefDefault = df_ProbaCoefDefault.drop(columns='index')

df_tempo = pd.DataFrame({'Ci_Parent':np.arange(1,101)})
df_tempo = pd.concat([df_tempo] * 100)
df_tempo = df_tempo.reset_index()
df_tempo = df_tempo.drop(columns='index')
df_ProbaCoefDefault['Ci_Parent'] = df_tempo

df_ProbaCoefDefault.head(10)


,Ci_Child,Ci_Parent
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
5,1,6
6,1,7
7,1,8
8,1,9
9,1,10


In [11]:

len(df_ProbaCoefDefault)


10000

In [12]:

df_info.head()


,Country_Name,Country_Code,Population,Region,Income_Groupe,Gini,gdpppp,Coef_Elast
0,Albania,ALB,2947314.000,Europe & Central Asia,Upper middle income,32.141,7297.000,0.400
1,Argentina,ARG,40080160.000,Latin America & Caribbean,Upper middle income,46.525,13220.000,0.660
2,Armenia,ARM,2907618.000,Europe & Central Asia,Upper middle income,28.030,5611.000,0.400
3,Austria,AUT,8321496.000,Europe & Central Asia,High income,29.533,36193.000,0.400
4,Azerbaijan,AZE,8763400.000,Europe & Central Asia,Upper middle income,32.903,8101.000,0.400


In [13]:

len(df_info)


74

In [14]:

#On crée un dataframe avec la répartition d'individu par défaut, à savoir tous les couples possibles. ( parent 1 enfant 1, parent 1 enfant 2, etc.)
df_ProbaParPays = pd.DataFrame()

for i in df_info.index:
    d = df_ProbaCoefDefault.copy()
    g = pd.DataFrame({"Country_Name":[df_info.loc[i, 'Country_Name']] * len(df_ProbaCoefDefault)})
    d = pd.concat([g,d], sort=False, axis=1)
    df_ProbaParPays = df_ProbaParPays.append(d,ignore_index=True,sort=False)
    
df_ProbaParPays.head(15) 


,Country_Name,Ci_Child,Ci_Parent
0,Albania,1,1
1,Albania,1,2
2,Albania,1,3
3,Albania,1,4
4,Albania,1,5
5,Albania,1,6
6,Albania,1,7
7,Albania,1,8
8,Albania,1,9
9,Albania,1,10


Donc, ici, chacun de nos pays a bien chaque couple parent enfant possible.

Essayons maintenant d'optimiser un peut nos données pour la suite.

In [15]:

df_ProbaParPays.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740000 entries, 0 to 739999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Country_Name  740000 non-null  object
 1   Ci_Child      740000 non-null  int64 
 2   Ci_Parent     740000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 16.9+ MB


In [16]:

#Modification des type de nos variable en type plus "leger" et optimiser
df_ProbaParPays = df_ProbaParPays.astype({'Country_Name': 'category','Ci_Child': 'int8','Ci_Parent': 'int8'})


In [17]:

df_ProbaParPays.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740000 entries, 0 to 739999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   Country_Name  740000 non-null  category
 1   Ci_Child      740000 non-null  int8    
 2   Ci_Parent     740000 non-null  int8    
dtypes: category(1), int8(2)
memory usage: 2.1 MB


On peut voire que nous avons déjâ gagner une grosse quantité de place. 

Nous faisons la meme chose pour l'autre dataframe

In [18]:

df_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country_Name   74 non-null     object 
 1   Country_Code   74 non-null     object 
 2   Population     74 non-null     float64
 3   Region         74 non-null     object 
 4   Income_Groupe  74 non-null     object 
 5   Gini           74 non-null     float64
 6   gdpppp         74 non-null     float64
 7   Coef_Elast     74 non-null     float64
dtypes: float64(4), object(4)
memory usage: 4.8+ KB


In [19]:

lis= {'Country_Name' : 'category',
      'Country_Code' : 'category'
      , 'Population' : 'float32',
      'Region' : 'category',
      'Income_Groupe' : 'category',
      'Gini' : 'float32',
      'Coef_Elast' : 'float32',
      'gdpppp':'float32'}

df_info = df_info.astype(lis)

df_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Country_Name   74 non-null     category
 1   Country_Code   74 non-null     category
 2   Population     74 non-null     float32 
 3   Region         74 non-null     category
 4   Income_Groupe  74 non-null     category
 5   Gini           74 non-null     float32 
 6   gdpppp         74 non-null     float32 
 7   Coef_Elast     74 non-null     float32 
dtypes: category(4), float32(4)
memory usage: 8.3 KB


In [20]:

df_info[['Country_Name','Coef_Elast']].head()


,Country_Name,Coef_Elast
0,Albania,0.400
1,Argentina,0.660
2,Armenia,0.400
3,Austria,0.400
4,Azerbaijan,0.400


Pour la suite de notre optimisation nous avons separer notre dataframe en plusieur plus petit pour faire les calcules.
Execution assez longue attention

In [21]:

# #Calcule du cd par pays
df_info1 = df_info[df_info.index.isin(range(0,10))].copy()
df_info2 = df_info[df_info.index.isin(range(10,20))].copy()
df_info3 = df_info[df_info.index.isin(range(20,30))].copy()
df_info4 = df_info[df_info.index.isin(range(30,40))].copy()
df_info5 = df_info[df_info.index.isin(range(40,50))].copy()
df_info6 = df_info[df_info.index.isin(range(50,60))].copy()
df_info7 = df_info[df_info.index.isin(range(60,len(df_info)))].copy()

df_info1["Cd"] = df_info1.Coef_Elast.apply(f.calculeCdPays)
df_info2["Cd"] = df_info2.Coef_Elast.apply(f.calculeCdPays)
df_info3["Cd"] = df_info3.Coef_Elast.apply(f.calculeCdPays)
df_info4["Cd"] = df_info4.Coef_Elast.apply(f.calculeCdPays)
df_info5["Cd"] = df_info5.Coef_Elast.apply(f.calculeCdPays)
df_info6["Cd"] = df_info6.Coef_Elast.apply(f.calculeCdPays)
df_info7["Cd"] = df_info7.Coef_Elast.apply(f.calculeCdPays)

#~9min



In [22]:

#Voici une des parties de notre dataframe
df_info1


,Country_Name,Country_Code,Population,Region,Income_Groupe,Gini,gdpppp,Coef_Elast,Cd
0,Albania,ALB,2947314.000,Europe & Central Asia,Upper middle income,32.141,7297.000,0.400,"[[0.0809, 0.0498, 0.0422, 0.0369, 0.0358, 0.03..."
1,Argentina,ARG,40080160.000,Latin America & Caribbean,Upper middle income,46.525,13220.000,0.660,"[[0.1547, 0.0875, 0.0709, 0.0555, 0.0479, 0.04..."
2,Armenia,ARM,2907618.000,Europe & Central Asia,Upper middle income,28.030,5611.000,0.400,"[[0.0798, 0.05, 0.0411, 0.0373, 0.0358, 0.0327..."
3,Austria,AUT,8321496.000,Europe & Central Asia,High income,29.533,36193.000,0.400,"[[0.077, 0.054, 0.0434, 0.0389, 0.0341, 0.0284..."
4,Azerbaijan,AZE,8763400.000,Europe & Central Asia,Upper middle income,32.903,8101.000,0.400,"[[0.0763, 0.0519, 0.0414, 0.0385, 0.0305, 0.02..."
5,Belgium,BEL,10709973.000,Europe & Central Asia,High income,30.158,33561.000,0.400,"[[0.0745, 0.0501, 0.0466, 0.0396, 0.0304, 0.02..."
6,Bulgaria,BGR,7492561.000,Europe & Central Asia,Upper middle income,37.814,11993.000,0.400,"[[0.0727, 0.05, 0.0416, 0.0362, 0.035, 0.0327,..."
7,Belarus,BLR,9527985.000,Europe & Central Asia,Upper middle income,28.329,11651.000,0.400,"[[0.0761, 0.0512, 0.0426, 0.0363, 0.0346, 0.03..."
8,Bolivia,BOL,9721454.000,Latin America & Caribbean,Lower middle income,57.572,3950.000,0.660,"[[0.1584, 0.089, 0.0675, 0.055, 0.0488, 0.0425..."
9,Brazil,BRA,192030368.000,Latin America & Caribbean,Upper middle income,55.890,9559.000,0.660,"[[0.1566, 0.086, 0.0698, 0.0575, 0.0465, 0.039..."


In [23]:

df_info1[['Country_Name','Cd']].head()


,Country_Name,Cd
0,Albania,"[[0.0809, 0.0498, 0.0422, 0.0369, 0.0358, 0.03..."
1,Argentina,"[[0.1547, 0.0875, 0.0709, 0.0555, 0.0479, 0.04..."
2,Armenia,"[[0.0798, 0.05, 0.0411, 0.0373, 0.0358, 0.0327..."
3,Austria,"[[0.077, 0.054, 0.0434, 0.0389, 0.0341, 0.0284..."
4,Azerbaijan,"[[0.0763, 0.0519, 0.0414, 0.0385, 0.0305, 0.02..."


In [24]:

df_info1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Country_Name   10 non-null     category
 1   Country_Code   10 non-null     category
 2   Population     10 non-null     float32 
 3   Region         10 non-null     category
 4   Income_Groupe  10 non-null     category
 5   Gini           10 non-null     float32 
 6   gdpppp         10 non-null     float32 
 7   Coef_Elast     10 non-null     float32 
 8   Cd             10 non-null     object  
dtypes: category(4), float32(4), object(1)
memory usage: 7.1+ KB


In [25]:

df_ProbaParPays.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740000 entries, 0 to 739999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   Country_Name  740000 non-null  category
 1   Ci_Child      740000 non-null  int8    
 2   Ci_Parent     740000 non-null  int8    
dtypes: category(1), int8(2)
memory usage: 2.1 MB


In [26]:

df_ProbaParPays.head()


,Country_Name,Ci_Child,Ci_Parent
0,Albania,1,1
1,Albania,1,2
2,Albania,1,3
3,Albania,1,4
4,Albania,1,5


In [27]:

df_ProbaParPays_G1 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info1['Country_Name'])].copy()
df_ProbaParPays_G2 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info2['Country_Name'])].copy()
df_ProbaParPays_G3 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info3['Country_Name'])].copy()
df_ProbaParPays_G4 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info4['Country_Name'])].copy()
df_ProbaParPays_G5 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info5['Country_Name'])].copy()
df_ProbaParPays_G6 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info6['Country_Name'])].copy()
df_ProbaParPays_G7 = df_ProbaParPays[df_ProbaParPays['Country_Name'].isin(df_info7['Country_Name'])].copy()



In [28]:

lastCountry = "None"
cdLastCountry = []

for row in df_ProbaParPays_G1.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info1[df_info1['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G1.loc[row.Index, 'Proba'] = a
    
for row in df_ProbaParPays_G2.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info2[df_info2['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G2.loc[row.Index, 'Proba'] = a
    
for row in df_ProbaParPays_G3.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info3[df_info3['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G3.loc[row.Index, 'Proba'] = a

for row in df_ProbaParPays_G4.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info4[df_info4['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G4.loc[row.Index, 'Proba'] = a

for row in df_ProbaParPays_G5.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info5[df_info5['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G5.loc[row.Index, 'Proba'] = a
    
for row in df_ProbaParPays_G6.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info6[df_info6['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G6.loc[row.Index, 'Proba'] = a
    
for row in df_ProbaParPays_G7.itertuples():
    
    if row.Country_Name == lastCountry:
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
    else:
        lastCountry = row.Country_Name
        cdLastCountry = df_info7[df_info7['Country_Name'] == row.Country_Name]['Cd']
        cd = cdLastCountry
        a = cd.values[0][row.Ci_Parent-1,row.Ci_Child-1]
        
    df_ProbaParPays_G7.loc[row.Index, 'Proba'] = a
    
    

In [29]:

df_ProbaParPays_G1 = pd.concat([df_ProbaParPays_G1, df_ProbaParPays_G2], ignore_index=True)
df_ProbaParPays_G1 = pd.concat([df_ProbaParPays_G1, df_ProbaParPays_G3], ignore_index=True)
df_ProbaParPays_G1 = pd.concat([df_ProbaParPays_G1, df_ProbaParPays_G4], ignore_index=True)
df_ProbaParPays_G1 = pd.concat([df_ProbaParPays_G1, df_ProbaParPays_G5], ignore_index=True)
df_ProbaParPays_G1 = pd.concat([df_ProbaParPays_G1, df_ProbaParPays_G6], ignore_index=True)
df_ProbaParPays_G1 = pd.concat([df_ProbaParPays_G1, df_ProbaParPays_G7], ignore_index=True)

df_ProbaParPays_G1['Nb'] = (df_ProbaParPays_G1['Proba'] * 500).round(0)
df_ProbaParPays_G1 = df_ProbaParPays_G1.astype({'Nb':'Int8'})



In [30]:

#Copie de chacun de couple en fonction de leur nombre voulue
df_Mission4 = pd.DataFrame()
temp_df = []
for row in df_ProbaParPays_G1.itertuples(index=False):
    temp_df.extend([list(row)] * row.Nb)
    
df_Mission4 = pd.DataFrame(temp_df, columns=df_ProbaParPays_G1.columns)


In [31]:

df_ProbaParPays_G1.head()


,Country_Name,Ci_Child,Ci_Parent,Proba,Nb
0,Albania,1,1,0.081,40
1,Albania,1,2,0.049,24
2,Albania,1,3,0.043,22
3,Albania,1,4,0.040,20
4,Albania,1,5,0.035,17


In [32]:

df_Mission4[(df_Mission4['Country_Name']=="Albania") & (df_Mission4.Ci_Parent == 1) & (df_Mission4.Ci_Child == 1)].head()


,Country_Name,Ci_Child,Ci_Parent,Proba,Nb
0,Albania,1,1,0.081,40
1,Albania,1,1,0.081,40
2,Albania,1,1,0.081,40
3,Albania,1,1,0.081,40
4,Albania,1,1,0.081,40


In [33]:

# Nous avons bien le meme nombre de couple que celui souhaiter
len(df_Mission4[(df_Mission4['Country_Name']=="Albania") & (df_Mission4.Ci_Parent == 1) & (df_Mission4.Ci_Child == 1)])


40

In [34]:

df_info.head()


,Country_Name,Country_Code,Population,Region,Income_Groupe,Gini,gdpppp,Coef_Elast
0,Albania,ALB,2947314.000,Europe & Central Asia,Upper middle income,32.141,7297.000,0.400
1,Argentina,ARG,40080160.000,Latin America & Caribbean,Upper middle income,46.525,13220.000,0.660
2,Armenia,ARM,2907618.000,Europe & Central Asia,Upper middle income,28.030,5611.000,0.400
3,Austria,AUT,8321496.000,Europe & Central Asia,High income,29.533,36193.000,0.400
4,Azerbaijan,AZE,8763400.000,Europe & Central Asia,Upper middle income,32.903,8101.000,0.400


In [35]:
df_p7.head()

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.898,7297.000
1,ALB,2008,2,100,916.662,7297.000
2,ALB,2008,3,100,1010.916,7297.000
3,ALB,2008,4,100,1086.908,7297.000
4,ALB,2008,5,100,1132.700,7297.000


In [36]:
df_p7.isna().sum()

country         0
year_survey     0
quantile        0
nb_quantiles    0
income          0
gdpppp          0
dtype: int64

In [37]:

#Preparation des données pour la question 4, comme demander en fin de question 3

df_M4 = pd.merge(right=df_Mission4, left=df_info, right_on='Country_Name', left_on='Country_Name', how='right')
# 
listColumnDrop = ['Region', 'Income_Groupe','gdpppp', 'Coef_Elast', 'Proba', 'Nb']

df_M4 = df_M4.drop(columns = listColumnDrop)
df_M4 = df_M4[['Country_Name', 'Country_Code','Ci_Child','Ci_Parent','Population','Gini']]

df_M4 = pd.merge(right=df_M4, left=df_p7, right_on=['Country_Code','Ci_Child'], left_on=['country','quantile'], how='right')

df_M4 = df_M4.drop(columns=["country","year_survey","quantile","nb_quantiles"])
df_M4 = df_M4[['Country_Code','Country_Name','Ci_Child','Ci_Parent','income','gdpppp','Population','Gini']]
df_M4 = df_M4.rename(columns={'income':'Income_Child'})
df_M4 = df_M4.dropna()
df_M4.head()


,Country_Code,Country_Name,Ci_Child,Ci_Parent,Income_Child,gdpppp,Population,Gini
0,ALB,Albania,1,1,728.898,7297.000,2947314.000,32.141
1,ALB,Albania,1,1,728.898,7297.000,2947314.000,32.141
2,ALB,Albania,1,1,728.898,7297.000,2947314.000,32.141
3,ALB,Albania,1,1,728.898,7297.000,2947314.000,32.141
4,ALB,Albania,1,1,728.898,7297.000,2947314.000,32.141


In [38]:

lis= {'Country_Code' : 'category',
      'Country_Name' : 'category',
      "Ci_Child":'int8',
      "Ci_Parent":'int8',
      'Income_Child' : 'float32',
      'gdpppp' : 'int16',
      'Population':'int32',
      'Gini':'float16'}

df_M4 = df_M4.astype(lis)
         
df_M4.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3699461 entries, 0 to 3699460
Data columns (total 8 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Country_Code  category
 1   Country_Name  category
 2   Ci_Child      int8    
 3   Ci_Parent     int8    
 4   Income_Child  float32 
 5   gdpppp        int16   
 6   Population    int32   
 7   Gini          float16 
dtypes: category(2), float16(1), float32(1), int16(1), int32(1), int8(2)
memory usage: 84.7 MB


In [39]:

#Export des données pour reutilisation.
df_M4.to_csv('data/clean/set_Mission4.csv',index=False)
